# Processing mixed beverage data (new)
This Jupyter Notebook uses [data.texas.gov Mixed Beverage Gross Receipts data](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893), and then a python library called [agate](http://agate.readthedocs.io/) to clean and process that data for [stories similar to this one](http://www.mystatesman.com/business/austin-alcohol-sales-percent-february/Oo2txZUkuDlqBl0rU9O1lJ/) on monthly alcohol sales.

This is a work in progress.

- The first version will use a downloaded file to process. The data is in a new format, so it has to be reworked
- The second phase will work on pulling the data directly from Socrata.

### This links may or may not work
- [Top sales statewide](#Top-sales-statewide)
- [Austin sales](#Austin-sales-and-sums)
- [Central Texas cities](#More-Central-Texas-cities)



I'll use a little bash here to look at the file I have pulled, which is files with `Obligation End Date` between Sept. 1 and Sept 30 2017.


In [1]:
%%bash
head -n 5 data-raw/Mixed_Beverage_Gross_Receipts-201709.csv

Taxpayer Number,Taxpayer Name,Taxpayer Address,Taxpayer City,Taxpayer State,Taxpayer Zip,Taxpayer County,Location Number,Location Name,Location Address,Location City,Location State,Location Zip,Location County,Inside/Outside City Limits,TABC Permit Number,Responsibility Begin Date,Responsibility End Date,Obligation End Date,Liquor Receipts,Wine Receipts,Beer Receipts,Cover Charge Receipts,Total Receipts
32052221093,"CTWL, LLC",2712 BEE CAVES RD STE 124,AUSTIN,TX,78746,227,1,CHINATOWN,2712 BEE CAVES RD STE 124,ROLLINGWOOD,TX,78746,227,Y,MB866922,03/20/2014,,09/30/2017,$8727,$12270,$2940,$0,$23937
32050880221,"ISB ENTERPRISE, LLC.",3659 NASA PKWY,SEABROOK,TX,77586,101,2,SIGNATURES,3659 NASA PKWY,SEABROOK,TX,77586,101,Y,MB851753,09/01/2016,,09/30/2017,$3710,$5067,$3582,$0,$12359
32040734306,"MAMA IRMA'S, INC.",7325 SPENCER HWY,PASADENA,TX,77505,101,1,MAMA IRMA'S,7325 SPENCER HWY,PASADENA,TX,77505,101,Y,MB764780,01/04/2011,,09/30/2017,$46186,$420,$18875,$0,$65481
32036703190,CAFE HOLDINGS 

This data appears to be MUCH cleaner than what we have worked with in the past. It looks like we won't have to trim fields.

Now that we have our file and know what it looks like, we'll use Python and the agate library to clean and analyze it. You'll need to make sure that you have agate installed, preferably in an virtual environment like Conda, as described in the [ReadMe](README.md).

In [2]:
# imports the libraries we will use
import agate
from decimal import Decimal
import re

/Users/christian/anaconda/envs/mixbev/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-12-02 11:42:13.610651. Please add timezone info to timestamps.


In [3]:
# this surpresses the timezone warning
# Might comment out during development so other warnings
# are not surpressed
import warnings
warnings.filterwarnings('ignore')

### Study variables

This is where you set which file you are working with, and which month you want to study, etc.

First, we'll list the files in our directory that we have downloaded so far so we can get the filename:

In [4]:
ls data-raw/

Mixed_Beverage_Gross_Receipts-201709.csv
Mixed_Beverage_Gross_Receipts-201710.csv


Then we set some values based on those.

- The **`file`** is the name of the file we want to process
- The **`tax_rate`** is the value we need for this file to get the Gross Recipts (vs the Tax Reported, which is just the tax amount the establishment paid). The comptroller [has information on the tax](https://comptroller.texas.gov/taxes/mixed-beverage/receipts.php), but this [old record layout](https://github.com/utdata/cli-tools/blob/master/data/mixbevtax/OLD-MIXEDBEVTAX-LAYOUT.txt) best describes the math.
- The **`month_studied`** is the YYYY/MM designation for the month before the file release. The file released in February has mostly records from January, but can also have any other month, so we set here the specific month we want. Note there is a check later on that counts the number of files by month, which is worth checking.

In [5]:
# this is our source file, which may have been downloaded above
file = 'data-raw/Mixed_Beverage_Gross_Receipts-201710.csv'

# Sets the tax rate to convert Report Tax to Gross Receipts
# It's 6.7 since January 1, 2014
tax_rate = Decimal('6.7')

# setting the month_studied var.
# This should be checked in the table below that counts records by month
# I DONT NEED THIS RIGHT NOW SINCE I'VE DOWNLOADED ONLY A SINGLE MONTH
month_studied = '2017/03'


### Import the file
There are a couple of things we have to set to import a file. Remember we had to do the same when we did this manually in Tableau.
- Set the column header names (BUT NEW DATA HAS HEADERS)
- Set the ZIP and County codes as text, so we preserve '001'.
- The encoding type of the file (Tableau enferred, but we have to specify here. Common types are 'iso-8859-1' or  'utf-8', or 'latin1'. Just try it until it works.

In [6]:
#Helps us import some text fields that may be considered numbers in error.
specified_types = {
    'Taxpayer Number': agate.Text(),
    'Location Number': agate.Text(),
    'Taxpayer Zip': agate.Text(),
    'Location Zip': agate.Text(),
    'Location County': agate.Text(),
    'Taxpayer County': agate.Text()
}

# this imports the file specified above, along with the proper types
mixbev_raw = agate.Table.from_csv(file, column_types=specified_types)

# prints table fields so we an check thoes data types
print(mixbev_raw)

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [7]:
# printing table to see results
mixbev_raw.print_table()


| Taxpayer Number | Taxpayer Name        | Taxpayer Address     | Taxpayer City | Taxpayer State | Taxpayer Zip | ... |
| --------------- | -------------------- | -------------------- | ------------- | -------------- | ------------ | --- |
| 12317173321     | ARAMARK EDUCATION... | PO BOX 7548          | PHILADELPHIA  | PA             | 19101        | ... |
| 32046754944     | ANA LUCIA VILLALTA   | 4736 MAPLE AVE       | DALLAS        | TX             | 75219        | ... |
| 17519330587     | CHILI'S BEVERAGE ... | 6820 LYNDON B JOH... | DALLAS        | TX             | 75240        | ... |
| 32053628460     | MOONSHINE DRINKER... | 2211 NORFOLK ST S... | HOUSTON       | TX             | 77098        | ... |
| 32021273860     | SUGARTIME LOUNGE INC | PO BOX 23176         | SAN ANTONIO   | TX             | 78223        | ... |
| 32056220901     | LOCAL GRINGOS, LLC   | 5911 MARQUITA AVE    | DALLAS        | TX             | 75206        | ... |
| 17518726561     | I GOTCHA, INC.      

## no cleaning?
- Unlike befor, we don't need to trim fields or calculate the Gross Receipts from Reported Tax, becaues the are already gross
- If we are interested in Tax paid, we will have to create a function to get those based on the tax rate, which will vary by the date of the report (there are two, which are outlined on the record layout linked from the [data primer](https://data.texas.gov/Government-and-Taxes/Mixed-Beverage-Gross-Receipts/naix-2893)

### Create establishment column

We do this so we make sure we have single establishments instead of grouping trade names together from different addresses, like 'CHILI'S BAR & GRILL'.

In [8]:
# Concatenates the name and address
mixbev_establishment = mixbev_raw.compute([
    ('Establishment', agate.Formula(agate.Text(), lambda row: '%(Location Name)s (%(Location Address)s)' % row))
])

# Prints columns so you see it is there
print(mixbev_establishment)

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [9]:
# selects and prints Establishment to check what is looks like
mixbev_establishment.select('Establishment').limit(5).print_table(max_column_width=80)

| Establishment                                                     |
| ----------------------------------------------------------------- |
| ARAMARK EDUCATIONAL SERVICES OF TEXAS, LLC (3300 DYER ST STE 101) |
| CANTINA EL CUBETAZO (4736 MAPLE AVE)                              |
| CHILI'S GRILL & BAR (2700 WOOD RIDGE BLVD)                        |
| MOONSHINE DRINKERY (103 W SANTA ROSA ST)                          |
| SUGARTIME (2730 BILL MILLER LN)                                   |


### Import and merge counties lookup table
We do this to get county names. I got this list from the comptroller.

NOTE: Wisdom would suggest we join on the `code` column from counites, but the data.texas.gov data does not have the zero padding from those values, so I'm using the `id` column.

In [10]:
# importing countes.csv, ensuring that the 'code' column is text
counties = agate.Table.from_csv('counties.csv', column_types={
    'code': agate.Text(),
    'id': agate.Text()
})

# peek at the column names
print(counties)

| column | data_type |
| ------ | --------- |
| id     | Text      |
| county | Text      |
| code   | Text      |



In [11]:
# peek at the data
counties.limit(5).print_table()
print(counties)

| id | county   | code |
| -- | -------- | ---- |
| 1  | Anderson | 001  |
| 2  | Andrews  | 002  |
| 3  | Angelina | 003  |
| 4  | Aransas  | 004  |
| 5  | Archer   | 005  |
| column | data_type |
| ------ | --------- |
| id     | Text      |
| county | Text      |
| code   | Text      |



In [12]:
# joines the counties table to the mixed bev cleaned data with establishments
mixbev_joined = mixbev_establishment.join(counties, 'Location County', 'id')

# check that the merge was succesful 
print(mixbev_joined)
mixbev_joined.print_table()

| column                     | data_type |
| -------------------------- | --------- |
| Taxpayer Number            | Text      |
| Taxpayer Name              | Text      |
| Taxpayer Address           | Text      |
| Taxpayer City              | Text      |
| Taxpayer State             | Text      |
| Taxpayer Zip               | Text      |
| Taxpayer County            | Text      |
| Location Number            | Text      |
| Location Name              | Text      |
| Location Address           | Text      |
| Location City              | Text      |
| Location State             | Text      |
| Location Zip               | Text      |
| Location County            | Text      |
| Inside/Outside City Limits | Boolean   |
| TABC Permit Number         | Text      |
| Responsibility Begin Date  | Date      |
| Responsibility End Date    | Date      |
| Obligation End Date        | Date      |
| Liquor Receipts            | Number    |
| Wine Receipts              | Number    |
| Beer Rece

In [13]:
# get just the columns we need and rename county
# THIS is the finished, cleaned mixbev table
mixbev = mixbev_joined.select([
    'Location Name',
    'Location Address',
    'Establishment',
    'Location City',
    'Location State',
    'Location Zip',
    'county',
    'Total Receipts',
    'Obligation End Date'
]).rename(column_names = {
    'Location Name' : 'Name',
    'Location Address' : 'Address',
    'Location City': 'City',
    'Location State': 'State',
    'Location Zip': 'Zip',
    'Total Receipts' : 'Receipts',
    'county': 'County',
    'Obligation End Date': 'Report date'
})

# peek at the column names
print(mixbev)

| column        | data_type |
| ------------- | --------- |
| Name          | Text      |
| Address       | Text      |
| Establishment | Text      |
| City          | Text      |
| State         | Text      |
| Zip           | Text      |
| County        | Text      |
| Receipts      | Number    |
| Report date   | Date      |



In [14]:
# peek at the table
mixbev.limit(5).print_table()

| Name                 | Address              | Establishment        | City            | State | Zip   | ... |
| -------------------- | -------------------- | -------------------- | --------------- | ----- | ----- | --- |
| ARAMARK EDUCATION... | 3300 DYER ST STE 101 | ARAMARK EDUCATION... | UNIVERSITY PARK | TX    | 75205 | ... |
| CANTINA EL CUBETAZO  | 4736 MAPLE AVE       | CANTINA EL CUBETA... | DALLAS          | TX    | 75219 | ... |
| CHILI'S GRILL & BAR  | 2700 WOOD RIDGE BLVD | CHILI'S GRILL & B... | BRENHAM         | TX    | 77833 | ... |
| MOONSHINE DRINKERY   | 103 W SANTA ROSA ST  | MOONSHINE DRINKER... | VICTORIA        | TX    | 77901 | ... |
| SUGARTIME            | 2730 BILL MILLER LN  | SUGARTIME (2730 B... | SAN ANTONIO     | TX    | 78223 | ... |


## No need to view dates

Previously we had to filter down to only the dates studied. As this stands now, we filtered to the correct date in Socrata BEFORE we exported. This might need to change once we pull from the API.

In [15]:
# double-checking I'm looking at one report data
mixbev_dates = mixbev.select('Report date').distinct('Report date')
mixbev_dates.print_table()

print('\nNumber of records in table: {}'.format(
        len(mixbev))
     )

| Report date |
| ----------- |
|  2017-10-31 |

Number of records in table: 14753


## Top sales statewide

Because we want to group our results by more than one field and perform more than one aggregation, we'll do this a little differently. We'll use group_by to create a grouped table, then perform aggregations on that new table to computer the Tax and Receipts columns.

In [17]:
# groups the data based on Establishment and City
mixbev_grouped = mixbev.group_by('Establishment').group_by('County').group_by('City')

# computes the sales based on the grouping
state_summary = mixbev_grouped.aggregate([
    ('Sales_sum', agate.Sum('Receipts'))
])

# sorts the results by most sold. We could probalby chain it above if we wanted to.
state_summary_sorted = state_summary.order_by('Sales_sum', reverse=True)

# prints the top 10 results
state_summary_sorted.limit(10).print_table(max_column_width=60)

| Establishment                                                | County  | City        | Sales_sum |
| ------------------------------------------------------------ | ------- | ----------- | --------- |
| RYAN SANDERS SPORTS SERVICES, LLC (9201 CIRCUIT OF THE AM... | Travis  | DEL VALLE   | 2,904,593 |
| AT&T STADIUM (1 LEGENDS WAY)                                 | Tarrant | ARLINGTON   | 2,819,059 |
| GAYLORD TEXAN (1501 GAYLORD TRL)                             | Tarrant | GRAPEVINE   | 1,807,271 |
| ARAMARK SPORTS & ENTERTAINMENT SERVICES OF TEXAS (501 CRA... | Harris  | HOUSTON     | 1,617,452 |
| SALC, INC. (2201 N STEMMONS FWY FL 1)                        | Dallas  | DALLAS      | 1,555,285 |
| HOSPITALITY INTERNATIONAL, INC. (23808 RESORT PKWY)          | Bexar   | SAN ANTONIO | 1,347,636 |
| ARAMARK SPORTS & ENTERTAINMENT SER OF TEXAS INC (8355 KIR... | Harris  | HOUSTON     | 1,210,323 |
| OMNI DALLAS CONVENTION CENTER (555 S LAMAR ST)               | Dallas  | DALLAS      | 1,

## Overall statewide sum

In [18]:
# summing sales statewide for month

print('Totalstatewide sales for this month are: {}'.format(
    mixbev.aggregate(agate.Sum('Receipts'))
))



Totalstatewide sales for this month are: 565676285


## Location sums function

Because we want to get the top sellers in a bunch of cities and couties, we create a function so we don't have to repeat the code. This function allows us to pass in a city or county name to filter the monthly receipts table and then sum the Tax and Receipts columns. The result can then be acted on to print or aggreggate.

In [19]:
# function to group sales by a specific location
# City or County passed in should be ALL CAPS
# Location_type can be 'City' or 'County'

def location_sum(location_type, location):
    # Filters the data to the specified city
    location_filtered = mixbev.where(lambda row: row[location_type].upper() == location)

    # groups the data based on Establishment and location
    location_grouped = location_filtered.group_by('Establishment').group_by(location_type)
    # computes the sales based on the grouping
    location_summary = location_grouped.aggregate([
        ('Receipts_sum', agate.Sum('Receipts'))
    ])
    
    # sorts the results by most sold
    location_summary_sorted = location_summary.order_by('Receipts_sum', reverse=True)
    # prints the top 10 results
    
    return(location_summary_sorted)


## Austin sales and sums

With this, we refernce the location_sum function above, and pass the type of location (City) and the name of the city (AUSTIN). At the same time, we limit the result of that function to the first 10 records, and then print the results. We are basically stringing together a bunch of stuff at once.

In [20]:
# uses the city_sum function to filter
austin = location_sum('City', 'AUSTIN')

# print the resulting table
austin.limit(5).print_table(max_column_width=60)

| Establishment                                                | City   | Receipts_sum |
| ------------------------------------------------------------ | ------ | ------------ |
| WLS BEVERAGE CO (110 E 2ND ST)                               | AUSTIN |    1,178,366 |
| BLIND PIG PUB / PIG PEN (317 E 6TH ST)                       | AUSTIN |      613,631 |
| 400 BAR/CUCARACHA/CHUPACABRA/JACKALOPE/MOOSENUCKLE (400 E... | AUSTIN |      543,720 |
| W HOTEL AUSTIN (200 LAVACA ST)                               | AUSTIN |      471,173 |
| ROSE ROOM/ 77 DEGREES (11500 ROCK ROSE AVE)                  | AUSTIN |      464,844 |


### String methods together to print a table
So far, we've been printing tables, but Agate can also print charts. The **`print_bars`** method creates a simple, text-based bar chart.

In [21]:
# We'll use the same function, but instead of creating a new table,
# we'll just string on the limit and print bars methods
# print_bars needs to arguments, the label column and then the value to make the chart from
location_sum('City', 'AUSTIN').limit(10).print_bars('Establishment', 'Receipts_sum', width=90)

Establishment                                                     Receipts_sum
WLS BEVERAGE CO (110 E 2ND ST)                                       1,178,366 ▓░░░░░░    
BLIND PIG PUB / PIG PEN (317 E 6TH ST)                                 613,631 ▓░░░       
400 BAR/CUCARACHA/CHUPACABRA/JACKALOPE/MOOSENUCKLE (400 E 6TH ST)      543,720 ▓░░░       
W HOTEL AUSTIN (200 LAVACA ST)                                         471,173 ▓░░        
ROSE ROOM/ 77 DEGREES (11500 ROCK ROSE AVE)                            464,844 ▓░░        
SALC. INC.(HILTON HOTEL) (500 E 4TH ST)                                431,178 ▓░░        
THE PALAZIO (501 E BEN WHITE BLVD)                                     430,533 ▓░░        
STUBB'S BAR-B-Q (801 RED RIVER ST)                                     402,490 ▓░░        
THE DOGWOOD DOMAIN (11420 ROCK ROSE AVE STE 700)                       401,258 ▓░░        
CULINARY DROPOUT (11721 ROCK ROSE AVE STE Y4.100)                      373,495 ▓░░        
           

### Total sales Austin

In [22]:
# Austin total sales as s city
# This sums the grouped table, but it works

print('Total sales in Austin are: {}'.format(
    location_sum('City', 'AUSTIN').aggregate(agate.Sum('Receipts_sum'))
))

Total sales in Austin are: 64614616


## More Central Texas cities

In [23]:
location_sum('City', 'BASTROP').limit(5).print_table(max_column_width=60)

| Establishment                                                | City    | Receipts_sum |
| ------------------------------------------------------------ | ------- | ------------ |
| OLD TOWN RESTURANT AND BAR/PINEY CREEK CHOP HOUSE (931 MA... | BASTROP |       68,015 |
| CHILI'S GRILL & BAR (734 HIGHWAY 71 W)                       | BASTROP |       51,414 |
| NEIGHBOR'S (601 CHESTNUT ST UNIT C)                          | BASTROP |       39,112 |
| BACK 9 (834 HIGHWAY 71 W)                                    | BASTROP |       35,749 |
| THE 602 ON MAIN (919 MAIN ST)                                | BASTROP |       27,938 |


In [24]:
location_sum('City', 'BEE CAVE').limit(3).print_table(max_column_width=60)

| Establishment                                         | City     | Receipts_sum |
| ----------------------------------------------------- | -------- | ------------ |
| WOODY TAVERN AND GRILL, INC. (12801 SHOPS PKWY # 100) | BEE CAVE |      119,842 |
| HCG BEVERAGE, LLC (12525 BEE CAVE PKWY)               | BEE CAVE |       89,746 |
| MAUDIE'S HILL COUNTRY, LLC (12506 SHOPS PKWY)         | BEE CAVE |       80,255 |


In [25]:
location_sum('City', 'BUDA').limit(3).print_table(max_column_width=60)

| Establishment                                               | City | Receipts_sum |
| ----------------------------------------------------------- | ---- | ------------ |
| WILLIE'S JOINT (824 MAIN ST)                                | BUDA |       92,781 |
| BUCKS BACKYARD (1750 S FM 1626)                             | BUDA |       55,582 |
| NATE'S AT BUDA MILL AND GRAIN, INC. (306 S MAIN ST STE 101) | BUDA |       52,176 |


In [26]:
location_sum('City', 'CEDAR PARK').limit(3).print_table(max_column_width=60)

| Establishment                                                | City       | Receipts_sum |
| ------------------------------------------------------------ | ---------- | ------------ |
| THE GROVE WINE BAR AND KITCHEN (1310 E WHITESTONE BLVD # ... | CEDAR PARK |      146,565 |
| CHUY'S (4911 183A TOLL RD)                                   | CEDAR PARK |      111,575 |
| PLUCKERS WING BAR (11066 PECAN PARK BLVD STE G)              | CEDAR PARK |      100,934 |


In [27]:
location_sum('City', 'DRIPPING SPRINGS').limit(3).print_table(max_column_width=60)

| Establishment                                             | City             | Receipts_sum |
| --------------------------------------------------------- | ---------------- | ------------ |
| TREATY OAK BREWING AND DISTILLING CO. (16604 FITZHUGH RD) | DRIPPING SPRINGS |       67,637 |
| TRUDY'S FOUR STAR (13059 FOUR STAR BLVD)                  | DRIPPING SPRINGS |       65,010 |
| FLORES MEXICAN RESTAURANT (2440 E HIGHWAY 290 BLDG D)     | DRIPPING SPRINGS |       57,566 |


In [28]:
location_sum('City', 'GEORGETOWN').limit(3).print_table(max_column_width=60)

| Establishment                              | City       | Receipts_sum |
| ------------------------------------------ | ---------- | ------------ |
| EL MONUMENTO (205 W 2ND ST)                | GEORGETOWN |       82,834 |
| GATEWAY BEVERAGES, INC (1101 WOODLAWN AVE) | GEORGETOWN |       73,446 |
| DOS SALSAS CAFE INC (1104 S MAIN ST)       | GEORGETOWN |       66,860 |


In [29]:
location_sum('City', 'KYLE').limit(3).print_table(max_column_width=60)

| Establishment                                           | City | Receipts_sum |
| ------------------------------------------------------- | ---- | ------------ |
| CASA GARCIA'S MEXICAN RESTAURANT (5401 FM 1626 STE 300) | KYLE |       68,336 |
| CENTERFIELD SPORTS BAR & GRILL (200 W CENTER ST)        | KYLE |       41,290 |
| EVO ENTERTAINMENT CENTER (3200 KYLE XING)               | KYLE |       38,776 |


In [30]:
location_sum('City', 'LAGO VISTA').limit(3).print_table(max_column_width=60)

| Establishment                                         | City       | Receipts_sum |
| ----------------------------------------------------- | ---------- | ------------ |
| THE GNARLY GAR (18200 LAKEPOINT CV)                   | LAGO VISTA |       45,615 |
| THE GRILLE AT HIGHLAND LAKES (20552 HIGHLAND LAKE DR) | LAGO VISTA |       18,417 |
| COPPERHEAD GRILL (6115 LOHMANS FORD RD)               | LAGO VISTA |       18,136 |


In [31]:
location_sum('City', 'LAKEWAY').limit(3).print_table(max_column_width=60)

| Establishment                                          | City    | Receipts_sum |
| ------------------------------------------------------ | ------- | ------------ |
| LAKEWAY RESORT AND SPA (101 LAKEWAY DR)                | LAKEWAY |      122,046 |
| THE GROVE WINE BAR AND KITCHEN (3001 RANCH ROAD 620 S) | LAKEWAY |      114,445 |
| THE HILLS AT LAKEWAY (26 CLUB ESTATES PKWY)            | LAKEWAY |      105,234 |


In [32]:
location_sum('City', 'LEANDER').limit(3).print_table(max_column_width=60)

| Establishment                                          | City    | Receipts_sum |
| ------------------------------------------------------ | ------- | ------------ |
| BROOKLYN HEIGHTS PIZZERIA (3550 LAKELINE BLVD STE 135) | LEANDER |       66,322 |
| JARDIN DEL REY (703 S HIGHWAY 183)                     | LEANDER |       37,707 |
| TAPATIA JALISCO #3 LLC (651 N US 183)                  | LEANDER |       12,443 |


In [33]:
location_sum('City', 'LIBERTY HILL').limit(3).print_table(max_column_width=60)

| Establishment                                        | City         | Receipts_sum |
| ---------------------------------------------------- | ------------ | ------------ |
| JARDIN CORONA (15395 W STATE HIGHWAY 29)             | LIBERTY HILL |       39,688 |
| MARGARITA'S RESTAURANT (10280 W STATE HIGHWAY 29)    | LIBERTY HILL |       34,860 |
| ELENAS MEXICAN RESTAURANT (14801 W STATE HIGHWAY 29) | LIBERTY HILL |        6,084 |


In [34]:
location_sum('City', 'PFLUGERVILLE').limit(3).print_table(max_column_width=60)

| Establishment                                  | City         | Receipts_sum |
| ---------------------------------------------- | ------------ | ------------ |
| MAVERICKS (1700 GRAND AVENUE PKWY STE 240)     | PFLUGERVILLE |      118,474 |
| HANOVER'S DRAUGHT HAUS (108 E MAIN ST)         | PFLUGERVILLE |       72,874 |
| RIO GRANDE TEX MEX (15821 CENTRAL COMMERCE DR) | PFLUGERVILLE |       54,604 |


In [35]:
location_sum('City', 'ROUND ROCK').limit(5).print_table(max_column_width=60)

| Establishment                                             | City       | Receipts_sum |
| --------------------------------------------------------- | ---------- | ------------ |
| THIRD BASE ROUND ROCK, LLC (3107 S INTERSTATE 35 STE 810) | ROUND ROCK |      154,984 |
| TWIN PEAKS RESTAURANT (100 LOUIS HENNA BLVD)              | ROUND ROCK |      149,813 |
| RICK'S CABARET (3105 S INTERSTATE 35)                     | ROUND ROCK |      138,443 |
| JACK ALLEN'S KITCHEN (2500 HOPPE TRL)                     | ROUND ROCK |      131,938 |
| PLUCKERS WING BAR (300 MAYS XING STE 300)                 | ROUND ROCK |      117,160 |


In [36]:
location_sum('City', 'SAN MARCOS').limit(5).print_table(max_column_width=60)

| Establishment                                              | City       | Receipts_sum |
| ---------------------------------------------------------- | ---------- | ------------ |
| PLUCKERS WING BAR (105 N INTERSTATE 35)                    | SAN MARCOS |      121,763 |
| THE MARC (120 E SAN ANTONIO ST)                            | SAN MARCOS |      120,155 |
| ZELICKS (336 W HOPKINS ST)                                 | SAN MARCOS |      110,193 |
| 54TH STREET RESTAURANT & DRAFTHOUSE (1303 S INTERSTATE 35) | SAN MARCOS |       92,174 |
| CHIMY'S SAN MARCOS (217 E HOPKINS ST)                      | SAN MARCOS |       91,857 |


In [37]:
location_sum('City', 'SPICEWOOD').limit(3).print_table(max_column_width=60)

| Establishment                                        | City      | Receipts_sum |
| ---------------------------------------------------- | --------- | ------------ |
| ANGEL'S ICEHOUSE (21815 W HWY 71)                    | SPICEWOOD |       58,696 |
| POODIES HILLTOP ROADHOUSE (22308 STATE HIGHWAY 71 W) | SPICEWOOD |       44,614 |
| J5 STEAKHOUSE (21814 STATE HIGHWAY 71 W)             | SPICEWOOD |       22,367 |


In [38]:
location_sum('City', 'SUNSET VALLEY').limit(3).print_table(max_column_width=60)

| Establishment                                          | City          | Receipts_sum |
| ------------------------------------------------------ | ------------- | ------------ |
| DOC'S BACKYARD (5207 BRODIE LN # 100)                  | SUNSET VALLEY |       89,901 |
| BJ'S RESTAURANT AND BREWHOUSE (5207 BRODIE LN STE 300) | SUNSET VALLEY |       64,383 |
| CANE ROSSO (4715 S LAMAR BLVD STE 103)                 | SUNSET VALLEY |       30,800 |


In [39]:
location_sum('City', 'WEST LAKE HILLS').limit(3).print_table(max_column_width=60)

| Establishment                                                | City            | Receipts_sum |
| ------------------------------------------------------------ | --------------- | ------------ |
| LUPE TORTILLA MEXICAN RESTAURANT (701 S CAPITAL OF TEXAS ... | WEST LAKE HILLS |       90,865 |
| CHIPOTLE CHIPOTLE MEXICAN GRILL CHIPOTLE BURRITOS (3300 B... | WEST LAKE HILLS |          743 |


## Sales by county

In this case, we pass in the location type of 'County' and then a county name in caps to get the most sales in a particular county.

In [40]:
location_sum('County', 'BASTROP').limit(5).print_table(max_column_width=80)

| Establishment                                                   | County  | Receipts_sum |
| --------------------------------------------------------------- | ------- | ------------ |
| LOST PINES BEVERAGE LLC (575 HYATT LOST PINES ROAD)             | Bastrop |      572,969 |
| OLD TOWN RESTURANT AND BAR/PINEY CREEK CHOP HOUSE (931 MAIN ST) | Bastrop |       68,015 |
| CHILI'S GRILL & BAR (734 HIGHWAY 71 W)                          | Bastrop |       51,414 |
| NEIGHBOR'S (601 CHESTNUT ST UNIT C)                             | Bastrop |       39,112 |
| BACK 9 (834 HIGHWAY 71 W)                                       | Bastrop |       35,749 |


In [41]:
location_sum('County', 'CALDWELL').limit(3).print_table(max_column_width=80)

| Establishment                                               | County   | Receipts_sum |
| ----------------------------------------------------------- | -------- | ------------ |
| EL REY BAR/NIGHTCLUB (1721 S COMMERCE ST)                   | Caldwell |       32,103 |
| GUADALAJARA MEXICAN RESTAURANT (1710 S COLORADO ST STE 110) | Caldwell |       15,866 |
| THE PEARL (110 N MAIN ST)                                   | Caldwell |       12,249 |


In [42]:
location_sum('County', 'HAYS').limit(3).print_table(max_column_width=80)

| Establishment                           | County | Receipts_sum |
| --------------------------------------- | ------ | ------------ |
| PLUCKERS WING BAR (105 N INTERSTATE 35) | Hays   |      121,763 |
| THE MARC (120 E SAN ANTONIO ST)         | Hays   |      120,155 |
| ZELICKS (336 W HOPKINS ST)              | Hays   |      110,193 |


In [43]:
location_sum('County', 'TRAVIS').limit(3).print_table(max_column_width=80)

| Establishment                                                         | County | Receipts_sum |
| --------------------------------------------------------------------- | ------ | ------------ |
| RYAN SANDERS SPORTS SERVICES, LLC (9201 CIRCUIT OF THE AMERICAS BLVD) | Travis |    2,904,593 |
| WLS BEVERAGE CO (110 E 2ND ST)                                        | Travis |    1,178,366 |
| BLIND PIG PUB / PIG PEN (317 E 6TH ST)                                | Travis |      613,631 |


In [44]:
location_sum('County', 'WILLIAMSON').limit(3).print_table(max_column_width=80)

| Establishment                                                 | County     | Receipts_sum |
| ------------------------------------------------------------- | ---------- | ------------ |
| TWIN PEAKS RESTAURANT (100 LOUIS HENNA BLVD)                  | Williamson |      149,813 |
| THE GROVE WINE BAR AND KITCHEN (1310 E WHITESTONE BLVD # 500) | Williamson |      146,565 |
| ALAMO DRAFTHOUSE CINEMA (14028 N HIGHWAY 183 BLDG F)          | Williamson |      145,688 |


## Sales by ZIP Code
For this, we'll do a simple pivot on Zip, but instead of using the default Count method, we'll pass in an aggregation to add the Tax values together for all the Zips. Whe then order_by in reverse to get the top values.

In [45]:
# top zip code gross receipts
zip_receipts = mixbev.pivot('Zip', aggregation=agate.Sum('Receipts')).order_by('Sum', reverse=True)
zip_receipts.limit(5).print_table()

| Zip   |        Sum |
| ----- | ---------- |
| 78701 | 25,150,987 |
| 75201 | 13,006,593 |
| 78205 |  9,100,329 |
| 77002 |  7,677,477 |
| 76011 |  7,500,736 |
